In [12]:
import os
import shutil
import csv
from collections import defaultdict
import utils
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [8]:
# Download the audio sets from https://zenodo.org/records/1290750#.WzCwSRyxXMU
# Please create a folder called /data and put all the extracted folders inside

def load_testset(dest):
    src = os.path.abspath('./data/IRMAS-TrainingData')

    file_dest = os.path.abspath(dest)

    print("Processing Traning Data")
    print(src)
    for root, _, files in os.walk(src):
        for file in files:
            if file[-4:] != '.wav':
                continue
            file_src = os.path.join(root, file)
            shutil.move(file_src, file_dest)

    print("Finished Moving Training Data")


load_testset('./TrainingData')

Processing Traning Data
f:\MiiD\data\IRMAS-TrainingData
Finished Moving Training Data


In [15]:
label_counts = defaultdict(int)
training_data = os.path.abspath('./TrainingData')

data = []
for _, _, files in os.walk(training_data):
    for file in files:
        label = utils.extract_anno(file)
        data.append({'Musical': label[0], 'Type': label[1]})
df = pd.DataFrame(data)
